In [33]:
#Emircan Ağaç 190401038

In [34]:
import pandas as pd
import torch.nn as nn
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score , precision_score , recall_score
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
# Define function to train the model
def train(model, optimizer, criterion, train_loader, val_loader, num_epochs=500, early_stop=None, epoch_display=10):
    best_val_loss = np.inf
    patience = 0
    for epoch in range(num_epochs):
        # Train for one epoch
        model.train()
        train_loss = 0.0
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(x_batch)
            loss = criterion(y_pred, y_batch.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        # Evaluate on validation set
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                y_pred = model(x_batch)
                val_loss += criterion(y_pred, y_batch.unsqueeze(1)).item()
            val_loss /= len(val_loader)
        
        #Add the training and validation losses to an array.
        list_train_loss.append(train_loss)
        list_val_loss.append(val_loss)

        # Early stopping if validation loss does not improve for early_stop consecutive epochs
        if early_stop is not None and val_loss < best_val_loss:
            best_val_loss = val_loss
            patience = 0
        elif early_stop is not None:
            patience += 1
            if patience == early_stop:
                print(f"Stopped training after {epoch} epochs. Best validation loss: {best_val_loss:.4f}")
                break

        # Print loss for every (Default=10) epochs
        if epoch % epoch_display == 0:
              print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

In [36]:
# Load the data
train_df = pd.read_csv("/content/cure_the_princess_train.csv")
val_df = pd.read_csv("/content/cure_the_princess_validation.csv")
test_df = pd.read_csv("/content/cure_the_princess_test.csv")

# Split into features and target
x_train = train_df.drop('Cured', axis=1).values
y_train = train_df['Cured'].values
x_val = val_df.drop('Cured', axis=1).values
y_val = val_df['Cured'].values
x_test = test_df.drop('Cured', axis=1).values
y_test = test_df['Cured'].values

In [37]:
# Convert to tensors and create datasets and dataloaders
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_val = torch.tensor(x_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [48]:
# Define the model architecture
class MLP(nn.Module):
    def __init__(self, input_dim,dropout_prob):
        super().__init__()
        self.fc1 = nn.Linear(13, 100)
        self.dropout1 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(100, 50)
        self.dropout2 = nn.Dropout(dropout_prob)
        self.fc3 = nn.Linear(50, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.sigmoid(self.fc3(x))
        return x
    
    def l2_regularization(self, weight_decay):
        l2_reg = None
        for name, param in self.named_parameters():
            if name.endswith("weight"):
                if l2_reg is None:
                    l2_reg = param.norm(2)**2
                else:
                    l2_reg = l2_reg + param.norm(2)**2

        return weight_decay*l2_reg

In [49]:
torch.manual_seed(190401038)  # Set the seed value to school student number

# Initialize the model and optimizer
model = MLP(input_dim=x_train.shape[1],dropout_prob=0.5)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001 ,weight_decay=0.01)
criterion = nn.BCELoss()

list_train_loss = []  
list_val_loss = []  

# Train the model
train(model, optimizer, criterion, train_loader, val_loader, num_epochs=500,early_stop=20)

Epoch 0, Train Loss: 0.9100, Validation Loss: 0.6742
Epoch 10, Train Loss: 0.6720, Validation Loss: 0.6167
Epoch 20, Train Loss: 0.6399, Validation Loss: 0.5626
Epoch 30, Train Loss: 0.6002, Validation Loss: 0.5164
Epoch 40, Train Loss: 0.5782, Validation Loss: 0.4743
Epoch 50, Train Loss: 0.5372, Validation Loss: 0.4149
Epoch 60, Train Loss: 0.5224, Validation Loss: 0.3859
Epoch 70, Train Loss: 0.4961, Validation Loss: 0.3486
Epoch 80, Train Loss: 0.4615, Validation Loss: 0.3237
Epoch 90, Train Loss: 0.4440, Validation Loss: 0.3003
Epoch 100, Train Loss: 0.4331, Validation Loss: 0.2844
Epoch 110, Train Loss: 0.4100, Validation Loss: 0.2643
Epoch 120, Train Loss: 0.3841, Validation Loss: 0.2514
Epoch 130, Train Loss: 0.3786, Validation Loss: 0.2429
Epoch 140, Train Loss: 0.3636, Validation Loss: 0.2306
Epoch 150, Train Loss: 0.3342, Validation Loss: 0.2221
Epoch 160, Train Loss: 0.3408, Validation Loss: 0.2163
Epoch 170, Train Loss: 0.3294, Validation Loss: 0.2132
Epoch 180, Train Loss

In [51]:
# Evaluate on test set
model.eval()
y_pred = []
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        y_pred_batch = model(x_batch)
        y_pred_batch = (y_pred_batch >= 0.5).float()
        y_pred.append(y_pred_batch)
y_pred = torch.cat(y_pred, dim=0).numpy()

acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Test Accuracy: {acc:.4f}, Test F1 Score: {f1:.4f}, Test Precision:{precision:.4f}, Test Recall:{recall:.4f}")

Test Accuracy: 0.9352, Test F1 Score: 0.9340, Test Precision:0.9568, Test Recall:0.9124
